In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI()

a = llm.predict("how many planets are there")
b = chat.predict("how many planets are there")

c:\Users\wmk51\FULLSTACK-GPT\env\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\wmk51\FULLSTACK-GPT\env\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [7]:
a,b


('There are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

##3-2

In [10]:
from langchain.chat_models import ChatOpenAI
#chatmodel 생성자
chat = ChatOpenAI(temperature=0.1) ##얼마나 창의적인 답변을 요구하느ㄴ냐

In [12]:

messages = chatPromptTemplate.from_messages[
    
]

chat.predict_messages(messages)

AIMessage(content='서울과 부산 사이의 거리는 약 325km입니다. 제 이름은 원호입니다. 어떻게 도와드릴까요?')

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# #PromptTemplate은 String을 가지고만 사용한다.
# template = PromptTemplate.from_template(
#     "What is the distance between {country_a} and {country_b}",
# )

# chat = ChatOpenAI(temperature=0.1)

# prompt = template.format(country_a = "Mexico", country_b = "Korea")
# chat.predict(prompt)

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
#PromptTemplate은 String을 가지고만 사용한다.
chat = ChatOpenAI(temperature=0.1)

#template은 큰 틀이고
template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a geography expert. And you only reply in {language}."),
        ("ai","Ciao, mi chiamao {name}!"),
        ("human", "what is the distance between {country_a} and {country_b}. Also, what is your name?")
    ]
)
#prompt 조건을 만족시키게 넣어주는 역활이다
prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [12]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
    
p = CommaOutputParser()
p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [16]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

###chain 만들기 위해서 필용한 요소 chat,outputParser,template
chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


template = ChatPromptTemplate.from_messages(
    [
        ("system","you are a generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}\
         in lowercase. Do NOT reply with anything else.",),
        ("human","{question}")
    ]
)





In [17]:
## "|"만 있으면 모든지 해결된다

chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items":5,
    "question": "what are the pokemons"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(temperature=0.1,
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()]
                  )
##첫 번쨰 chain 요리사
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients"),
    ("human","I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

In [23]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
      ("system","You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe.\
          If there is no alternative for a food just say you don't know how to replace it.")  ,
      ("human","{recipe}")
    ]
)

veg_chain = veg_chef_prompt | chat
## 코드 해석
## chef_chain의 결과값을 recipe라하고 veg_chain에 대입한다
final_chain = {"recipe" : chef_chain} | veg_chain
final_chain.invoke({"cuisine" :"indian" })

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's make a classic dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 2 tbsp tomato paste
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1 tsp paprika
- 1 tsp garam masala
- 1/2 tsp turmeric
- Salt and pepper, to taste
- Fresh cilantro, for garnish
- Cooked rice or naan, for serving

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tbsp of vegetable oil, half of the minced garlic, half of the grated ginger, and a pinch of salt. Add the chicken pieces and marinate for at least 1 hour, preferably overnight.

2. Preheat the oven to 400°F (200°C). Thread the marinated chicken onto skewers and place them on

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, we can replace the chicken with a suitable alternative such as paneer or tofu. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb paneer or extra-firm tofu, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan version)\n- 2 tbsp lemon juice\n- 2 tbsp vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) crushed tomatoes\n- 2 tbsp tomato paste\n- 1 tsp ground cumin\n- 1 tsp ground coriander\n- 1 tsp paprika\n- 1 tsp garam masala\n- 1/2 tsp turmeric\n- Salt and pepper, to taste\n- Fresh cilantro, for garnish\n- Cooked rice or naan, for serving\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but use paneer or tofu instead of chicken. Marinate the paneer or tofu in the yogurt, lemon juice, vegetable oil, garlic, ginger, and salt for at least 1 hour.\n\n2. Instead of bak

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
chat = ChatOpenAI(temperature=0.1,
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()]
                  )


examples = [
    {
        "question" : "what do you know about France",
        "answer" : """
        Here is what I know:
        Captial: Paris
        Language: French
        Food : Wine and Cheese
        Currency: Euro
        """,
        
    },
    {
        "question" : "what do you know about Italy",
        "answer" : """
        Here is what I know:
        Captial: Rome
        Language: Italian
        Food : Pizza and Pasta
        Currency: Euro
        """,
        
    },
    {
        "question" : "what do you know about Greek",
        "answer" : """
        Here is what I know:
        Captial: Athens
        Language: Greek
        Food : Souvlaki and Feta Cheese
        Currency: Euro
        """,
        
    },
]


example_template = """
    Human:{question}
    AI:{answer}
"""

example_prompt = PromptTemplate.from_template("Human: {question}\n AI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt= example_prompt,
    examples=examples,
    suffix="Human : what do you know about {country}",
    input_variables=["country"]
)

prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({
    "country" : "Turkey"
})

AI: 
        Here is what I know:
        Capital: Ankara
        Language: Turkish
        Food: Kebab and Baklava
        Currency: Turkish Lira

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebab and Baklava\n        Currency: Turkish Lira')

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate


chat = ChatOpenAI(temperature=0.1,
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()]
                  )


examples = [
    {
        "country" : "France",
        "answer" : """
        Here is what I know:
        Captial: Paris
        Language: French
        Food : Wine and Cheese
        Currency: Euro
        """,
        
    },
    {
        "country" : "Italy",
        "answer" : """
        Here is what I know:
        Captial: Rome
        Language: Italian
        Food : Pizza and Pasta
        Currency: Euro
        """,
        
    },
    {
        "country" : "Greek",
        "answer" : """
        Here is what I know:
        Captial: Athens
        Language: Greek
        Food : Souvlaki and Feta Cheese
        Currency: Euro
        """,
        
    },
]


example_template = """
    Human:{question}
    AI:{answer}
"""

example_prompt = ChatPromptTemplate.from_messages(
    [
    ("human","what do you know about {country}?"),
    ("ai","{answer}")
    ]
    )

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt= example_prompt,
    examples=examples,
    
)

final_prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a geography expert,  you give short answers."),
    example_prompt,
    ("human","what do you know about {country}?")]
)

chain = final_prompt | chat

chain.invoke({
    "country" : "Germany"
})


        Here is what I know:
        Captial: Berlin
        Language: German
        Food : Bratwurst and Sauerkraut
        Currency: Euro
        

AIMessageChunk(content='\n        Here is what I know:\n        Captial: Berlin\n        Language: German\n        Food : Bratwurst and Sauerkraut\n        Currency: Euro\n        ')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector


chat = ChatOpenAI(temperature=0.1,
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()]
                  )


examples = [
    {
        "country" : "France",
        "answer" : """
        Here is what I know:
        Captial: Paris
        Language: French
        Food : Wine and Cheese
        Currency: Euro
        """,
        
    },
    {
        "country" : "Italy",
        "answer" : """
        Here is what I know:
        Captial: Rome
        Language: Italian
        Food : Pizza and Pasta
        Currency: Euro
        """,
        
    },
    {
        "country" : "Greek",
        "answer" : """
        Here is what I know:
        Captial: Athens
        Language: Greek
        Food : Souvlaki and Feta Cheese
        Currency: Euro
        """,
        
    },
]


example_template = """
    Human:{question}
    AI:{answer}
"""

example_prompt = ChatPromptTemplate.from_messages(
    [
    ("human","what do you know about {country}?"),
    ("ai","{answer}")
    ]
    )

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt= example_prompt,
    examples=examples,
    
)

final_prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a geography expert,  you give short answers."),
    example_prompt,
    ("human","what do you know about {country}?")]
)

chain = final_prompt | chat

chain.invoke({
    "country" : "Germany"
})

In [7]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector



chat = ChatOpenAI(temperature=0.1,
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()]
                  )


examples = [
    {
        "question" : "what do you know about France",
        "answer" : """
        Here is what I know:
        Captial: Paris
        Language: French
        Food : Wine and Cheese
        Currency: Euro
        """,
        
    },
    {
        "question" : "what do you know about Italy",
        "answer" : """
        Here is what I know:
        Captial: Rome
        Language: Italian
        Food : Pizza and Pasta
        Currency: Euro
        """,
        
    },
    {
        "question" : "what do you know about Greek",
        "answer" : """
        Here is what I know:
        Captial: Athens
        Language: Greek
        Food : Souvlaki and Feta Cheese
        Currency: Euro
        """,
        
    },
]

class RandomExampleSelector(BaseExampleSelector):
    
    def __init__(self,examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]


example_template = """
    Human:{question}
    AI:{answer}
"""

example_prompt = PromptTemplate.from_template("Human: {question}\n AI:{answer}")

#설정한 max_length만큼의 예제를 학습시킬 수 있다.
#random으로 가져오지 않을 경우
#example_selector = LengthBasedExampleSelector(
#    examples = examples,
#    example_prompt=example_prompt,
#   max_length=180,
#)

example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt=example_prompt,
    max_length=180,
)

prompt = FewShotPromptTemplate(
    example_prompt= example_prompt,
    example_selector=example_selector,
    suffix="Human : what do you know about {country}",
    input_variables=["country"]
)

prompt.format(country="Brazil")



'Human: what do you know about France\n AI:\n        Here is what I know:\n        Captial: Paris\n        Language: French\n        Food : Wine and Cheese\n        Currency: Euro\n        \n\nHuman: what do you know about Italy\n AI:\n        Here is what I know:\n        Captial: Rome\n        Language: Italian\n        Food : Pizza and Pasta\n        Currency: Euro\n        \n\nHuman : what do you know about Brazil'

###4-4 Serialization and Composition

In [13]:

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")
prompt.format(country="Germany")

chat = ChatOpenAI(temperature=0.1,
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()]
                  )


prompt.format(country="xx")


'What is the capital of xx'

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate
#이미 질문한 질문에 대한 답을 저장해서 비용 절약
from langchain.globals import set_llm_cache



chat = ChatOpenAI(temperature=0.1,
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()]
                  )

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro",intro),
    ("example",example),
    ("start",start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
                                     )

full_prompt.format(
    character="Pirate",
    example_question="what is your location?",
    example_answer="Arrrg! That is a secret!!",
    question="what is your favorite food?"
    
    
)

chain = full_prompt | chat

chain.invoke({
    "character":"Pirate",
    "example_question":"what is your location?",
    "example_answer":"Arrrg! That is a secret!!",
    "question":"what is your favorite food?"
})

Arrrg! Me favorite food be a hearty stew made with fresh seafood and plenty of spices! Aye, nothing beats a good meal after a long day of pillaging and plundering on the high seas!

AIMessageChunk(content='Arrrg! Me favorite food be a hearty stew made with fresh seafood and plenty of spices! Aye, nothing beats a good meal after a long day of pillaging and plundering on the high seas!')

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

#이미 질문한 질문에 대한 답을 저장해서 비용 절약
from langchain.globals import set_llm_cache,set_debug
from langchain.cache import InMemoryCache,SQLiteCache


set_llm_cache(InMemoryCache())
# set_llm_cache(SQLiteCache())
set_debug(True)


chat = ChatOpenAI(temperature=0.1,
                #   streaming=True,
                #   callbacks=[StreamingStdOutCallbackHandler()]
                  )

chat.predict("how do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: how do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [4.39s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a l

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [18]:
chat.predict("how do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'